In [1]:
import os
import json

from io import BytesIO

from urllib.parse import urlparse
from urllib.parse import parse_qs

import requests

import numpy as np
import pandas as pd

from PIL import Image
from jinja2 import Template

In [2]:
folder = '09-serverless'

In [3]:
os.makedirs(folder + '/images', exist_ok=True)

In [4]:
with open(folder + '/meta.json') as f_in:
    data = json.load(f_in)

In [5]:
session = data['session']
session_name = data['name']

In [6]:
df = pd.read_csv(folder + '/' + data['data'])

In [7]:
df = df.fillna('')

In [8]:
df

,lesson,name,page_name,video,slides,notebook
0,1,Introduction to Serverless,01-intro.md,https://www.youtube.com/watch?v=JLIVwIsU6RA,,
1,2,AWS Lambda,02-aws-lambda.md,https://www.youtube.com/watch?v=_UX8-2WhHZo,,
2,3,TensorFlow Lite,03-tensorflow-lite.md,https://www.youtube.com/watch?v=OzZA4mSBE0Q,,code/tensorflow-model.ipynb
3,4,Preparing the code for Lambda,04-preparing-code.md,https://www.youtube.com/watch?v=XXBUivsHhec,,
4,5,Preparing a Docker image,05-docker-image.md,https://www.youtube.com/watch?v=y4_YQjfOsDo,,
5,6,Creating the lambda function,06-creating-lambda.md,https://www.youtube.com/watch?v=kBch5oD5BkY,,
6,7,API Gateway: exposing the lambda function,07-api-gateway.md,https://www.youtube.com/watch?v=wyZ9aqQOXvs,,
7,8,Summary,08-summary.md,https://www.youtube.com/watch?v=bu3nPiHCNLU,,
8,9,Explore more,09-explore-more.md,,,
9,10,Homework,homework.md,,,


In [9]:
template_string = """
## {{ session }}.{{ lesson }} {{ name }}

{% if video %}<a href="{{ video }}"><img src="{{ thumbnail }}"></a>{% endif %}
{% if coming_soon %}COMING SOON{% endif %} 

{% if slides %}[Slides]({{ slides }}){% endif %}


## Notes

Add notes from the video (PRs are welcome)


<table>
   <tr>
      <td>⚠️</td>
      <td>
         The notes are written by the community. <br>
         If you see an error here, please create a PR with a fix.
      </td>
   </tr>
</table>


## Nagivation

{{ navigation }}
"""

In [10]:
def download_thumbnail(video, session, lesson):
    if type(lesson) in [np.int64]:
        thumbnail_file = 'thumbnail-%d-%02d.jpg' % (session, lesson)
    else:
        thumbnail_file = 'thumbnail-%d-%s.jpg' % (session, lesson)
    thumbnail_file_full = '%s/images/%s' % (folder, thumbnail_file)

    if os.path.exists(thumbnail_file_full):
        print(thumbnail_file_full, 'exists')
        return 'images/%s' % thumbnail_file

    video_id = parse_qs(urlparse(video).query)['v'][0]
    print('processing video', video_id)
    thumbnail_url = 'https://img.youtube.com/vi/%s/0.jpg' % video_id

    response = requests.get(thumbnail_url)
    thumbnail = Image.open(BytesIO(response.content))
    w_img, h_img = thumbnail.size

    play = Image.open('../images/play.png')
    w_play, h_play = play.size
    
    x0 = w_img // 2 - w_play // 2
    y0 = h_img // 2 - h_play // 2

    thumbnail.paste(play, (x0, y0), play)
    thumbnail.save(thumbnail_file_full, quality=90)

    print('saved to', thumbnail_file_full)

    return 'images/%s' % thumbnail_file

In [11]:
create_files = False

for i in range(len(df)):
    row = df.loc[i]
    params = row.to_dict()
    params['session'] = session

    navigation = [
        '* [Machine Learning Zoomcamp course](../)',
        '* [Session %s: %s](./)' % (session, session_name) 
    ]

    if i > 0:
        prev_row = df.loc[i - 1]
        line = '* Previous: [%s](%s)' % (prev_row['name'], prev_row.page_name)
        navigation.append(line)

    if i + 1 < len(df):
        next_row = df.loc[i + 1]
        line = '* Next: [%s](%s)' % (next_row['name'], next_row.page_name)
        navigation.append(line)
        
    params['navigation'] = '\n'.join(navigation)
    
    if row.video and row.video != 'NOT_READY':
        thumbnail = download_thumbnail(row.video, session, row.lesson)
        params['thumbnail'] = thumbnail

    if row.video == 'NOT_READY':
        del params['video']
        params['coming_soon'] = True
    
    if 'video' in row:
        params['video'] = params['video'] + '&list=PL3MmuxUbc_hIhxl5Ji8t4O6lPAOpHaCLR'

    template = Template(template_string)
    rendered = template.render(**params)
    print(rendered)
    
    page_full = '%s/%s' % (folder, row.page_name)
    
    print()

    if create_files:
        if (not os.path.exists(page_full)) or os.path.getsize(page_full) <= 1:
            with open(page_full, 'w') as f_out:
                f_out.write(rendered)
            print("created", page_full)
        else:
            with open(page_full, 'a') as f_out:
                f_out.write('\n\n========\n\n')
                f_out.write(rendered)
            print("APPENDED TO", page_full)
    
    print()
    print()

09-serverless/images/thumbnail-9-01.jpg exists

## 9.1 Introduction to Serverless

<a href="https://www.youtube.com/watch?v=JLIVwIsU6RA"><img src="images/thumbnail-9-01.jpg"></a>
 




## Notes

Add notes from the video (PRs are welcome)


<table>
   <tr>
      <td>⚠️</td>
      <td>
         The notes are written by the community. <br>
         If you see an error here, please create a PR with a fix.
      </td>
   </tr>
</table>


## Nagivation

* [Machine Learning Zoomcamp course](../)
* [Session 9: Serverless Deep Learning](./)
* Next: [AWS Lambda](02-aws-lambda.md)



09-serverless/images/thumbnail-9-02.jpg exists

## 9.2 AWS Lambda

<a href="https://www.youtube.com/watch?v=_UX8-2WhHZo"><img src="images/thumbnail-9-02.jpg"></a>
 




## Notes

Add notes from the video (PRs are welcome)


<table>
   <tr>
      <td>⚠️</td>
      <td>
         The notes are written by the community. <br>
         If you see an error here, please create a PR with a fix.
      </td>
   </tr>
</table>

